### Multiplane Holography

TODO



#### Initialization

To start, we initialize our system:

In [ ]:
# Header. This is hidden from sphinx with the json metadata:
#   {"nbsphinx":"hidden"}

# ipython configuration (reloads source code automatically and plots inline)
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os, sys
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('image', cmap='Blues')

# Add slmsuite to the python path (For the case where it isn't installed via pip).
# slmsuite-examples is assumed to be installed in the same directory as slmsuite.
sys.path.append(os.path.join(os.getcwd(), '../../slmsuite'))

from slmsuite.hardware.slms.santec import Santec
from slmsuite.hardware.cameras.alliedvision import AlliedVision
from slmsuite.hardware.cameraslms import FourierSLM

from slmsuite.holography.toolbox.phase import blaze, zernike_sum

In [ ]:
# **README**                        (This cell is hidden from readthedocs rendering)
# If you're reading this, you might have gotten an error from loading physical
# hardware in the next cell, but that doesn't mean you can't run this notebook!
# This cell loads virtual hardware. You can skip the next cell.
from slmsuite.hardware.slms.simulated import SimulatedSLM
from slmsuite.hardware.cameras.simulated import SimulatedCamera

# Make the SLM and camera.
slm = SimulatedSLM((1600, 1200), pitch_um=(8,8))
slm.set_source_analytic(sim=True)               # Program the virtual source.
slm.set_source_analytic()                       # Don't bother with wavefront calibration, instead just set it to the same source.

cam = SimulatedCamera(slm, (1440, 1100), pitch_um=(4,4), gain=50)

# Tie the camera and SLM together with an analytic Fourier calibration.
fs = FourierSLM(cam, slm)
M, b = fs.fourier_calibration_build(
    f_eff=80000.,                               # f_eff of 80000 wavelengths or 80 mm with the default 1 um wavelength
    theta=5 * np.pi / 180,                      # Slight rotation for fun.
)
fs.fourier_calibrate_analytic(M, b)

In [ ]:
slm = Santec(slm_number=1, display_number=2, wav_um=.633, settle_time_s=.5); print()
cam = AlliedVision(serial="02C5V", fliplr=True)
fs = FourierSLM(cam, slm)

In [ ]:
fs.fourier_calibrate(plot=True)

#### Planes of Focus

TODO

In [ ]:
path = os.path.join(os.getcwd(), 'docs/source/static/slmsuite-small.png')

h, w = banner_shape = (640, 1280)
x, y = banner_shift = (0, -225)

y0, x0 = np.array(cam.shape) / 2

x += x0
y += y0

i0,i1,i2,i3 = (int(y-h/2), int(y+h/2), int(x-w/2), int(x+w/2))
p = 5

print(i0,i1,i2,i3)

def load_img(path, target_shape=None, angle=0, shift=(-200, 0), plot=False):
    # Load the image.
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        raise ValueError("Image not found at path '{}'".format(path))

    # Invert if necessary such that the majority of the image is dark.
    if np.mean(img) > np.mean(cv2.bitwise_not(img)):
        img = cv2.bitwise_not(img)

    if angle != 0:
        img = ndimage.rotate(img, angle)

    if target_shape is not None:
        zoom_x = target_shape[0] / img.shape[0]
        zoom_y = target_shape[1] / img.shape[1]
        img = ndimage.zoom(img, min(zoom_x, zoom_y))

    # sqrt to get the amplitude.
    target_ij = toolbox.pad(np.sqrt(img.astype(np.float32) / 255), fs.cam.shape)

    # Shift to the desired center.
    target_ij = np.roll(target_ij, shift, axis=(0,1))

    if plot:
        plot_img(target_ij)
        plt.show()

    target_ij[:(i0-p), :] = np.nan
    target_ij[(i1+p):, :] = np.nan
    target_ij[:, :(i2-p)] = np.nan
    target_ij[:, (i3+p):] = np.nan

    return target_ij

def plot_img(target_ij):
    # Plot the desired camera space.
    plt.figure(figsize=(16,12))
    plt.imshow(target_ij)
    plt.plot(target_ij.shape[1]/2, target_ij.shape[0]/2, 'r*')

target_ij = load_img(path, shift=(-225, -170), plot=True);

banner = target_ij[i0:i1, i2:i3]

plt.figure(figsize=(16,12))
plt.imshow(banner)

plt.figure(figsize=(16,12))
plt.imshow(banner[96:-96])

In [ ]:
from slmsuite.holography.algorithms import MultiplaneHologram, FeedbackHologram

f_eff_target = 1e7
N = 2048
p=5

holograms = []
weights = []

for sym, f_eff in zip(["smith", "spin", "text"], [-f_eff_target, np.inf, f_eff_target]):
    target_ij = load_img(
        os.path.join(os.getcwd(), f'docs/source/static/slmsuite-small-{sym}.png'),
        shift=(-225, -170), plot=False
    );

    holograms.append(
        FeedbackHologram(
            (N, N), target_ij,
            null_region_radius_frac=.5, cameraslm=fs,
            propagation_kernel=lens(slm, f_eff)
        )
    )
    weights.append(np.sqrt(np.nansum(np.square(target_ij))))

mh = MultiplaneHologram(holograms=holograms, weights=weights)

In [ ]:
import tqdm

osc = (1/f_eff_target) * np.sin(np.linspace(0, 2*np.pi, 100))
cam.set_exposure(150)

imgs = []

for f in tqdm.tqdm(np.reciprocal(osc)):
    slm.write(mh.get_phase() + lens(slm, f))
    img_ij = cam.get_image()

    banner = img_ij[i0:i1, i2:i3]
    imgs.append(banner)

In [ ]:
from slmsuite.holography.analysis.files import write_image

write_image("mutiplane.test.v10.gif", imgs, cmap="Blues", normalize=False)

#### Planes of Basis

TODO

In [ ]:
from slmsuite.holography.algorithms import MultiplaneHologram, FeedbackHologram, CompressedSpotHologram
from slmsuite.holography.toolbox import convert_vector

holograms = []
weights = []

target_ij = load_img(
    os.path.join(os.getcwd(), f'docs/source/static/slmsuite-small.png'),
    shift=(-225, -170), plot=False
)

holograms.append(
    FeedbackHologram(
        (2048, 2048),
        target_ij,
        null_region_radius_frac=.5,
        cameraslm=fs
    )
)

N = 28
i = np.arange(N)
nl = zernike_convert_index(i, from_index="ansi", to_index="radial")
p = 27
x = 1175
y = cam.shape[0]/2 - 225

vectors_ij = np.hstack((
    np.vstack((x + p * nl[:, 1], y - p * (.5 + nl[:, 0]) * np.sqrt(3))),
    np.vstack((x + p * nl[:, 1], y + p * (.5 + nl[:, 0]) * np.sqrt(3)))
))

vectors_zernike_xy = convert_vector(
    vectors_ij,
    from_units="ij",
    to_units="zernike",
    hardware=fs
)

vectors_zernike = np.zeros((N,2*N))
vectors_zernike[2, :] = vectors_zernike_xy[0]
vectors_zernike[1, :] = vectors_zernike_xy[1]

perturbation = np.diag(np.ones(N))
perturbation = np.hstack((perturbation, perturbation))

vectors_zernike_pyramid = vectors_zernike + 5*perturbation

vectors_zernike_final = np.vstack((vectors_zernike_pyramid, np.zeros(2*N)))
vectors_zernike_final[-1, :N] = 2 # LG02
basis = np.hstack((i, [-1]))

holograms.append(
    CompressedSpotHologram(
        spot_vectors=vectors_zernike_final,
        basis=basis,
        cameraslm=fs
    )
)

mh = MultiplaneHologram(holograms=holograms, weights=[1, .0003])

In [ ]:
mh.reset_phase(random_phase=0, quadratic_phase=2)
mh.optimize(method="WGS-Leonardo", maxiter=10, mraf_factor=1, feedback_exponent=.8)

In [ ]:
slm.write(mh.get_phase())
cam.set_exposure(120)
img_ij = cam.get_image()

banner = img_ij[i0:i1, i2:i3]

plt.figure(figsize=(16,8))
plt.imshow(banner)
plt.show()

write_image("slmsuite-banner.png", banner)

#### Planes of Both

TODO

In [ ]:
from slmsuite.holography.algorithms import MultiplaneHologram

f_eff_target = 1e7
N = 2048

holograms = []
weights = []

for sym, depth in zip(["smith", "spin", "text"], [-f_eff_target, np.inf, f_eff_target]):
    target_ij = load_img(
        os.path.join(os.getcwd(), f'docs/source/static/slmsuite-small-{sym}.png'),
        shift=(-225, -170), plot=False
    );

    holograms.append(
        FeedbackHologram(
            (2048, 2048), target_ij,
            null_region_radius_frac=.5, cameraslm=fs,
            propagation_kernel=lens(slm, depth)
        )
    )

    weights.append(np.sqrt(np.nansum(np.square(target_ij))))


target_ij = load_img(
    os.path.join(os.getcwd(), f'docs/source/static/slmsuite-small.png'),
    shift=(-225, -170), plot=False
);

target_ij = cv2.GaussianBlur(target_ij, (31,31), 0)

# Points
N_spots = 200

spots_ij = np.vstack((
    np.random.randint(i2, i3, size=N_spots),
    np.random.randint(i0, i1, size=N_spots)
))

from slmsuite.holography.toolbox import lloyds_algorithm

spots_ij = lloyds_algorithm(
    _generate_grid(cam.shape[1], cam.shape[0]),
    spots_ij,
    iterations=1
).astype(int)

# Remove points which are close of the logo.
nooverlap = target_ij[spots_ij[1], spots_ij[0]] == 0
spots_ij = spots_ij[:, nooverlap]
N_spots = spots_ij.shape[1]

plt.imshow(target_ij)
plt.scatter(spots_ij[0], spots_ij[1])
plt.show()

# Convert to kxy units so we can add on depth in units of focal power.
spots_kxy = convert_vector(
    spots_ij,
    from_units="ij",
    to_units="kxy",
    hardware=fs
)

spots_kxyz = np.vstack((
    spots_kxy,
    4 * (np.random.rand(N_spots) - .5) / f_eff_target
))

# Finally, convert to zernike.
spots_zernike = convert_vector(
    spots_kxyz,
    from_units="kxy",
    to_units="zernike",
    hardware=fs
)

# Turn every spot into a bottle beam.
basis = [
    2,      # x
    1,      # y
    4,      # z
    -1      # LG10 - vortex waveplate
]

spots_zernike = np.vstack((
    spots_zernike,
    2 * np.ones((1, N_spots))   # LG20 on every spot, for fun.
))

# Make the spot hologram.
holograms.append(
    CompressedSpotHologram(
        spot_vectors=spots_zernike,
        basis=basis,
        cameraslm=fs
    )
)
weights.append(.05)

mh = MultiplaneHologram(holograms=holograms, weights=weights)

In [ ]:
mh.reset_phase(random_phase=0, quadratic_phase=2)
mh.optimize(method="GS", maxiter=5, mraf_factor=1)
mh.optimize(method="WGS-Leonardo", maxiter=20, mraf_factor=1)

In [ ]:
import tqdm

osc = (1/f_eff_target) * np.sin(np.linspace(0, 2*np.pi, 100))
cam.set_exposure(75)

imgs = []

for f in tqdm.tqdm(np.reciprocal(osc)):
    slm.write(mh.get_phase() + lens(slm, f))
    img_ij = cam.get_image()

    banner = img_ij[i0:i1, i2:i3]
    imgs.append(banner)

In [ ]:
from slmsuite.holography.analysis.files import write_image

# The .gifs are a bit large at raw resolution, so we downscale by 2x
N = 2
imgs = np.array(imgs)
imgs2 = 0

for x in range(N):
    for y in range(N):
        imgs2 += imgs[::4, x::N, y::N] // (N * N)

write_image("ex-slmsuite-3d.gif", imgs2, cmap="Blues", border=127, normalize=False)
write_image("ex-slmsuite-3d-dark.gif", imgs2, cmap="turbo", border=127, normalize=False)